In [154]:
import pandas as pd
from sklearn.cross_decomposition import PLSRegression
import pandas as pd
from sklearn.cross_decomposition import PLSRegression
import calendar
import numpy as np

#loading dataframes 

File_url_d = 'https://raw.githubusercontent.com/Sebasleen/Seminargroup/Seminar/managed_portfolios_anom_d_50.csv'

r_daily = pd.read_csv(File_url_d)

# drop all momentum factors or factors that are constructed based on momentum

r_daily['date'] = pd.to_datetime(r_daily['date'])
r_daily.set_index('date', inplace=True)

# select our Y variable of interest 

y_daily = r_daily['r_mom']
y_monthly = y_daily.resample('M').sum()
y_monthly.index = y_monthly.index.strftime('%Y-%m')


momentum_list = ['r_mom', 'r_indmom', 'r_valmom', 'r_valmomprof', 'r_mom12', 'r_momrev', 'r_indmomrev']



# following the procedure in the paper, if there are observations missing we set them to 0. 

r_daily.fillna(0, inplace=True)
r_daily.drop(columns=['rme', 're_ew','r_mom', 'r_indmom', 'r_valmom', 'r_valmomprof', 'r_mom12', 'r_momrev', 'r_indmomrev'], inplace=True)



# create a list of factors for later analysis purposes 

factors = [col for col in r_daily.columns if col.startswith('r_')]

 
# create a monthly return dataframe for later analysis purposes 

r_monthly = r_daily.resample('M').sum()
r_monthly.index = r_monthly.index.strftime('%Y-%m')




In [157]:


# select our start date 

start_date = pd.to_datetime("1963-07-01")

pls_avg_df = pd.DataFrame()

# create an empty list for the pls return dataframes. These will be concated in a later stage to one large dataframe

pls_return_dfs = []


# Create an empty dataframe for monthly PLS returns
pls_return_df = pd.DataFrame(columns=['date'] + [f"PLS{i+1}" for i in range(len(factors))])

# Initialize an empty list to store PLS weights
pls_weights_list = []

# Loop through years and months
for year in range(1973, 1975):
    # as our out of sample procedure starts July 1973, we start in the 7th month in 1973
    for mo in range(6,13) if year == 1973 else range(1, 13):
        # first we have to find the last month of the day. For this we use the calender function with inputs from the loop variables
        last_day = calendar.monthrange(year, mo)[1]

        # we select our new end_date variable for which the Pls analysis is done, also with inputs from our loop and the last_day variable
        end_date = pd.to_datetime(f'{year}-{mo}-{last_day}')

        t_dt = pd.to_datetime(f'{year}-{mo}')
        t = t_dt.strftime('%Y-%m')

        # Select data range for PLS analysis
        pls_data = r_daily.loc[start_date:end_date]
        pls_data_y = y_daily.loc[start_date:end_date]

        # Separate features (X) and target (y)
        X = pls_data
        y = pls_data_y

        # Fit PLS model
        pls = PLSRegression(n_components=43)  
        pls.fit(X, y)


        # Extract and store PLS weights
        pls_weights = pd.DataFrame(
            np.array(pls.x_weights_),  
            index=factors,
            columns=[f"PLS{i+1}" for i in range(43)] 
        )
        # calculating return for month t+1. If mo = 12, then year will increment with 1. 

        t_plus_1_year = year + 1 if mo == 12 else year
        t_plus_1_month = (mo % 12) + 1

        # creating a datetime variable for the month t+1 and storing this in our pls_return_data variable

        t_plus_1_dt =pd.to_datetime(f'{t_plus_1_year}-{t_plus_1_month}')
        t_plus_1 = t_plus_1_dt.strftime('%Y-%m')

        pls_return_data = {'date': t_plus_1}


        # in this loop we calculate the monthly factor returns (f) using the principal components and returns

        for f in range(len(factors)):
            # select our factor and extract its principal component from principal_df and its return from r_daily for all observations in month mo 
            pls = pls_weights.iloc[:, f]
            r_month = r_monthly.loc[t]

            # multiply the principal components with the returns and sum them up to get Pls factor return for month mo 
            pls_return = (pls*r_month).sum()

            # place this in our dictionary for later transposing to dataframe

            pls_return_data[pls_weights.columns[f]] = pls_return

            r_pls_month_n_list = []

            for n in range(1, 12):
                # calculate the datetime for t - n
                t_minus_n_dt = t_dt - pd.DateOffset(months=n)

                # transpose it to our YYYY-MM format
                t_minus_n = t_minus_n_dt.strftime('%Y-%m')

                # select the return corresponding to our month t-n
                r_month_n = r_monthly.shift(n).loc[t_minus_n]
                pls_return_n = (pls*r_month_n).sum()
                r_pls_month_n_list.append(pls_return_n)

            r_pls_month_mean = (np.mean(r_pls_month_n_list))
            pls_avg_df.loc[t, f'PLS{f+1}'] = r_pls_month_mean


        pls_return_df = pd.DataFrame.from_dict(pls_return_data, orient='index').T
        pls_return_df.set_index('date', inplace=True)
        pls_return_dfs.append(pls_return_df)

r_pls = pd.concat(pls_return_dfs)
print(r_pls)
print(pls_avg_df)


             PLS1      PLS2      PLS3      PLS4      PLS5      PLS6      PLS7  \
date                                                                            
1973-07 -0.055711  0.129687  0.037249 -0.024036 -0.020074 -0.021063   0.04419   
1973-08   0.02762  -0.50513 -0.212508  0.007123  0.097147 -0.138669  0.084288   
1973-09  -0.00516  0.054417  0.009852 -0.019354  0.007235  0.006986  0.028687   
1973-10 -0.022413 -0.085402  0.061641 -0.181277  0.038235  0.056513   0.03602   
1973-11 -0.000221  0.222399  0.037388 -0.108471 -0.005582  0.057491 -0.041453   
1973-12 -0.066438  0.400494  0.079694  0.031917 -0.188383 -0.252431 -0.153685   
1974-01 -0.015944  0.212297  0.014748 -0.173475 -0.120844 -0.020569  0.085818   
1974-02 -0.114794  0.054834 -0.254585 -0.472944  -0.05298   0.04567 -0.016058   
1974-03  0.017549  0.127884 -0.139296 -0.147162 -0.089391 -0.057346 -0.026781   
1974-04 -0.028002 -0.025034 -0.008806 -0.049435  -0.01166  0.008574 -0.065131   
1974-05   0.08597  0.041205 